This notebook gives a summary of the NIRISS commissioning data

In [1]:
from pathlib import Path
import pandas as pd
from astropy.io import fits

First, we just get a list of all simulation files.

In [2]:
# data_dir = Path(Path.home(), "cap_simulations")
data_dir = Path(Path.home(), "Documents/jwst_data/cap_sims/cap_simulations/")
data_files = list(data_dir.iterdir())

# Summary and example header
We can first take a look at the info and header of an example file.

In [3]:
hdul = fits.open(data_files[5])

In [4]:
# General info
hdul.info()

Filename: /home/vandal/Documents/jwst_data/cap_sims/cap_simulations/jw01093001001_01101_00006_nis_rateints.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     166   ()      
  1  SCI           1 ImageHDU        48   (80, 80, 65)   float32   
  2  ERR           1 ImageHDU        11   (80, 80, 65)   float32   
  3  DQ            1 ImageHDU        12   (80, 80, 65)   int32 (rescales to uint32)   
  4  INT_TIMES     1 BinTableHDU     24   0R x 7C   [J, D, D, D, D, D, D]   
  5  VAR_POISSON    1 ImageHDU        10   (80, 80, 65)   float32   
  6  VAR_RNOISE    1 ImageHDU        10   (80, 80, 65)   float32   
  7  ASDF          1 BinTableHDU     11   1R x 1C   [5069B]   


In [5]:
# Primary header
hdul[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
DATE    = '2021-05-07T15:48:14.955' / UTC date file created                     
ORIGIN  = 'STScI   '           / Organization responsible for creating file     
FILENAME= 'jw01093001001_01101_00006_nis_rateints.fits' / Name of the file      
FILETYPE= 'countrate'          / Type of data in the file                       
CAL_VER = '1.1.0   '           / Calibration software version number            
CAL_VCS = 'RELEASE '           / Calibration software version control sys number
DATAMODL= 'CubeModel'          / Type of data model                             
TELESCOP= 'JWST    '           / Telescope used to acquire the data             
                            

In [6]:
# SCI header
hdul[1].header

XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                  -32 / array data type                                
NAXIS   =                    3 / number of array dimensions                     
NAXIS1  =                   80                                                  
NAXIS2  =                   80                                                  
NAXIS3  =                   65                                                  
PCOUNT  =                    0 / number of parameters                           
GCOUNT  =                    1 / number of groups                               
EXTNAME = 'SCI     '           / extension name                                 
BUNIT   = 'DN/s    '           / physical units of the array values             
                                                                                
        Information about the coordinates in the file                           
                            

In [7]:
# Close the file
hdul.close()

The primary header has a lot of useful info on the observation itself and on
the instrument configuration (exposure, dither, object name, date, program, etc.).

I also printed the SCI header (hdul[1].header) because it seems important to orient the image properly
Here are a few useful links (I think):
- https://docs.astropy.org/en/stable/wcs/index.html
- https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html
- https://jwst-docs.stsci.edu/jwst-observatory-hardware/jwst-observatory-coordinate-system-and-field-of-regard
- https://jwst-docs.stsci.edu/jwst-observatory-functionality/jwst-position-angles-ranges-and-offsets

# Dataset summary
Now we can load all primary headers to get a summary of the observations.
We use a pandas dataframe to easily group and filter information.

In [8]:
d = []
for fpath in data_files:
    hdr = fits.getheader(fpath)
    # remove blank keys from header
    del hdr['']
    d.append(dict(hdr))

df = pd.DataFrame(d)

In [21]:
# Example quick filter
# df_filters = (df.SUBARRAY == "SUB80")
# df = df[df_filters]

# This grouby can be used to see how many files we have, broken down in
# detailed catergories
# We can comment categories if we want to simplify the summary
group_cat = [
    "OBSLABEL",
    "TARGPROP",
    "FILTER",
    "PUPIL",
    # "EXP_TYPE",  # Seems redundent with SUBARRAY in terms of classifying the dataset
    "SUBARRAY",
    "PATT_NUM",
    "SUBPXPTS",
    "XOFFSET",
    "YOFFSET",
    "NINTS",
    "NGROUPS"
]
cat_sizes = df.groupby(group_cat).size()
cat_sizes.to_frame(name="NFILES")

NFILES
OBSLABEL       TARGPROP            FILTER PUPIL  SUBARRAY PATT_NUM SUBPXPTS XOFFSET YOFFSET NINTS NGROUPS        
Observation 1  AB-DOR              F380M  NRM    SUB80    1        1         0.000   0.000  94    2             1
                                                          2        1        -1.831   1.127  94    2             1
                                   F430M  NRM    SUB80    1        1         0.000   0.000  96    3             1
                                                          2        1        -1.831   1.127  96    3             1
                                   F480M  NRM    SUB80    1        1         0.000   0.000  29    13            1
                                                                                            65    5             1
                                                          2        1        -1.831   1.127  29    13            1
                                                                                            65    5             1
                                                 SUBTAAMI 0        1        -0.196   0.329  1     7             1
                                                                            -0.131  -0.262  1     7             1
                                                                             0.000   0.000  1     7             2
Observation 10 CPD-66-562          F480M  CLEARP SUB80    1        1         0.000   0.000  240   13            1
                                                          2        1         0.294   0.294  240   13            1
                                          NRM    SUB80    1        1         0.000   0.000  10    10            1
                                                 SUBTAAMI 0        1        -0.196   0.329  1     19            1
                                                                            -0.131  -0.262  1     19            1
                                                                             0.000   0.000  1     19            2
Observation 11 CPD-67-607          F480M  CLEARP SUB80    1        1         0.000   0.000  240   13            1
                                                          2        1         0.294   0.294  240   13            1
                                          NRM    SUB80    1        1         0.000   0.000  10    10            1
                                                 SUBTAAMI 0        1        -0.196   0.329  1     19            1
                                                                            -0.131  -0.262  1     19            1
                                                                             0.000   0.000  1     19            1
Observation 2  AB-DOR              F380M  NRM    SUB80    1        1         0.000   0.000  94    2             1
                                                          2        1        -1.831   1.127  94    2             1
                                   F430M  NRM    SUB80    1        1         0.000   0.000  96    3             1
                                                          2        1        -1.831   1.127  96    3             1
                                   F480M  NRM    SUB80    1        1         0.000   0.000  65    5             1
                                                          2        1        -1.831   1.127  65    5             1
                                                 SUBTAAMI 0        1        -0.196   0.329  1     3             1
                                                                            -0.131  -0.262  1     3             1
                                                                             0.000   0.000  1     3             2
Observation 3  AB-DOR              F480M  NRM    SUB80    1        5        -0.009  -0.009  65    5             1
                                                                                     0.009  65    5             1
   

In [23]:
# Now we can use the above categories as index to get files for each of them
pd.set_option("display.max_rows", None)
cat_df = df.set_index(group_cat)
cat_files = cat_df.FILENAME
# cat_files.to_frame().reset_index()
cat_files.to_frame()

FILENAME
OBSLABEL       TARGPROP            FILTER PUPIL  SUBARRAY PATT_NUM SUBPXPTS XOFFSET YOFFSET NINTS NGROUPS                                             
Observation 1  AB-DOR              F480M  NRM    SUBTAAMI 0        1         0.000   0.000  1     7        jw01093001001_01101_00001_nis_rateints.fits
                                                                            -0.131  -0.262  1     7        jw01093001001_01101_00002_nis_rateints.fits
                                                                            -0.196   0.329  1     7        jw01093001001_01101_00003_nis_rateints.fits
                                                                             0.000   0.000  1     7        jw01093001001_01101_00004_nis_rateints.fits
                                                 SUB80    1        1         0.000   0.000  65    5        jw01093001001_01101_00005_nis_rateints.fits
                                                          2        1        -1.831   1.127  65    5        jw01093001001_01101_00006_nis_rateints.fits
                                   F430M  NRM    SUB80    1        1         0.000   0.000  96    3        jw01093001001_01101_00007_nis_rateints.fits
                                                          2        1        -1.831   1.127  96    3        jw01093001001_01101_00008_nis_rateints.fits
                                   F380M  NRM    SUB80    1        1         0.000   0.000  94    2        jw01093001001_01101_00009_nis_rateints.fits
                                                          2        1        -1.831   1.127  94    2        jw01093001001_01101_00010_nis_rateints.fits
                                   F480M  NRM    SUB80    1        1         0.000   0.000  29    13       jw01093001001_01101_00011_nis_rateints.fits
                                                          2        1        -1.831   1.127  29    13       jw01093001001_01101_00012_nis_rateints.fits
Observation 2  AB-DOR              F480M  NRM    SUBTAAMI 0        1         0.000   0.000  1     3        jw01093002001_01101_00001_nis_rateints.fits
                                                                            -0.131  -0.262  1     3        jw01093002001_01101_00002_nis_rateints.fits
                                                                            -0.196   0.329  1     3        jw01093002001_01101_00003_nis_rateints.fits
                                                                             0.000   0.000  1     3        jw01093002001_01101_00004_nis_rateints.fits
                                                 SUB80    1        1         0.000   0.000  65    5        jw01093002001_01101_00005_nis_rateints.fits
                                                          2        1        -1.831   1.127  65    5        jw01093002001_01101_00006_nis_rateints.fits
                                   F430M  NRM    SUB80    1        1         0.000   0.000  96    3        jw01093002001_01101_00007_nis_rateints.fits
                                                          2        1        -1.831   1.127  96    3        jw01093002001_01101_00008_nis_rateints.fits
                                   F380M  NRM    SUB80    1        1         0.000   0.000  94    2        jw01093002001_01101_00009_nis_rateints.fits
                                                          2        1        -1.831   1.127  94    2        jw01093002001_01101_00010_nis_rateints.fits
Observation 3  AB-DOR              F480M  NRM    SUBTAAMI 0        1         0.000   0.000  1     7        jw01093003001_01101_00001_nis_rateints.fits
                                                                            -0.131  -0.262  1     7        jw01093003001_01101_00002_nis_rateints.fits
                                                                            -0.196   0.329  1     7        jw01093003001_01101_00003_nis_rateints.fits
                                                                 

Some observations on the above file info:
- In Observation 1, The two files with 29 integrations per exposure in F480M at the end of observation 1 are used to reach the 72k limit, TA SNR is 286
- Observation 2 is the same as 1, but no extra F480M

Here's how the files are links (for the parts I think I understand):
- 1 and 2 have 4 and 6 as calibrators
- 3 has 5 and 7 as calibrators
    - Not sure why 7 has 25 subpixel dithers